### 스타벅스 매장 입지 분석
- 순서
    1. 데이터 수집
    2. 데이터 전처리
    3. 분석
    4. 시각화

#### 데이터 수집
- 셀레니움 자동화 + 뷰티풀수프 정제

In [1]:
# 필요 라이브러리 사용
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [2]:
!pip install matplotlib


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install seaborn


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install tqdm


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# 분석, 시각화 라이브러리 사용
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams, rc, font_manager
import warnings
import seaborn as sns
import time
from tqdm import tqdm # 반복 진행 프로그레스바 모듈

C:\Users\user\AppData\Local\Temp\ipykernel_19424\1646539710.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
# 맵플롯립 한글 및 기타설정

## 맑은 고딕 이름 가져오기
font_path = 'C:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=font_path).get_name() #Malgun Gothic
warnings.simplefilter('ignore') # 경고메세지 무시(출력숨김)

## 맷플롭립 설정
plt.rcParams['font.family'] = font_name # 폰트 글자체 설정
plt.rcParams['font.size'] = 12 # 글자크기
plt.rcParams['figure.figsize'] = (12, 6) # (넓이, 높이)
plt.rcParams['axes.grid'] = True # 차트 가로세로줄 표시
plt.rcParams['axes.unicode_minus'] = False # 한글 설정 후 마이너스깨짐 방지

## 시본 설정
sns.set_theme(font=font_name, style='darkgrid', rc={'axes.unicode_minus':False})

##### 셀레니움 사용

In [7]:
# 웹드라이버
driver = webdriver.Chrome()


In [8]:
# 스타벅스 코리아사이트 연결
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=quick'
driver.get(url)

In [9]:
# 매장찾기 웹소스에서 '지역 검색' 링크 검색 후 링크 클릭
## F12 개발자도구로 HTML 태그 정보 확인
## header.loca_search > h3 > a에 지역검색 링크 존재

link_path = 'header.loca_search > h3 > a' ## header 태그의 class 명 = loca_search
driver.find_element(By.CSS_SELECTOR, link_path).click()

In [10]:
## 지역 검색에서 시/도 클릭 (서울 클릭)
## ul.sido_arae_box > li:nth-child(1) > a
link_path = 'ul.sido_arae_box > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, link_path).click()

In [11]:
## 서울에서 전체 클릭
## ul.gugun_arae_box > li.nth-child(1) > a
link_path = 'ul.gugun_arae_box > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, link_path).click()

## 매장 전체 조회가 최초 2~3초 정도 시간이 걸리기 떄문에 딜레이를 걸어줌
time.sleep(3)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"ul.gugun_arae_box > li:nth-child(1) > a"}
  (Session info: chrome=127.0.6533.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF65AB39632+30946]
	(No symbol) [0x00007FF65AAEE3C9]
	(No symbol) [0x00007FF65A9E6FDA]
	(No symbol) [0x00007FF65AA3822C]
	(No symbol) [0x00007FF65AA3850C]
	(No symbol) [0x00007FF65AA7DCB7]
	(No symbol) [0x00007FF65AA5CAAF]
	(No symbol) [0x00007FF65AA7B041]
	(No symbol) [0x00007FF65AA5C813]
	(No symbol) [0x00007FF65AA2A6E5]
	(No symbol) [0x00007FF65AA2B021]
	GetHandleVerifier [0x00007FF65AC6F83D+1301229]
	GetHandleVerifier [0x00007FF65AC7BDB7+1351783]
	GetHandleVerifier [0x00007FF65AC72A03+1313971]
	GetHandleVerifier [0x00007FF65AB6DD06+245686]
	(No symbol) [0x00007FF65AAF758F]
	(No symbol) [0x00007FF65AAF3804]
	(

- 메감커피, 빽다방, 컴포즈 등 크롤링 가능한 사이트도 동일하게 진행

##### 서울 스타벅스 매장 정보 가져오기

In [ ]:
html = driver.page_source

In [ ]:
soup = BeautifulSoup(html,'html.parser')

In [ ]:
# 서울, 전체 결과에서
## ul.quickSearchResultBoxSidoGugun > li.auickResultLstCon 만 가져오면 됨
quickResultLst = soup.select('li.quickResultLstCon')
len(quickResultLst) 

637

In [ ]:
# 각 결과 확인
quickResultLst[31].select('i')[0]['class'][0].split('_')[1]

'reserve'

In [ ]:
# 서울매장 리스트 DF
seoulStores = []

for item in tqdm(quickResultLst): # 진행률 표시하기 위해 tqdm으로 감싸기
    storeName = item.select('strong')[0].text.strip() # 매장이름
    storeLat = item['data-lat'] # 매장 위도값
    storeLng = item['data-long'] # 매장 위치 경도값
    adress = item.select('p.result_details')[0].text.replace('1522-3232', '') # 매장 주소
    storeCd = item['data-storecd'] # 매장코드(번호가 작을수록 오래된 매장)
    storeType = item.select('i')[0]['class'][0].split('_')[1] # 매장 타입(일반, 리저브, DT, WT)

    seoulStores.append([storeCd, storeName, storeType, adress, storeLat, storeLng])

100%|██████████| 637/637 [00:00<00:00, 8979.51it/s]


In [ ]:
# 리스트 확인
len(seoulStores)

637

In [ ]:
# DF로 변환
df_seoulStore = pd.DataFrame(seoulStores, columns=['매장 코드', '매장명', '매장유형', '주소', '위도', '경도'])
df_seoulStore.head(2)

,매장 코드,매장명,매장유형,주소,위도,경도
0,1190,동명대DT,generalDT,부산광역시 남구 신선로 423 (용당동),35.12311959047579,129.09901642703608
1,2235,부산유엔공원,general,부산광역시 남구 유엔로 200 (대연동),35.1299808,129.0980971


In [ ]:
df_seoulStore.tail(3)

,매장 코드,매장명,매장유형,주소,위도,경도
634,1668,묵동,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
635,2002,양원역,general,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053
636,1749,중화역,general,서울특별시 중랑구 봉화산로 35 1층,37.60170912407773,127.07841136432036


In [ ]:
# 데이터 저장
df_seoulStore.to_csv('./data/스타벅스_매장정보원본.csv', encoding='utf-8')

##### 데이터 전처리
- 데이터로 통계를 낼 때 문제가 없도록 데이터를 전부 사전에 가공을 해주는 것
    - 필요없는 col 제거
    - 필요한데 없는 col 추가
    - 잘못된 값 변경
    - 데이터 결측치(= 수치값이 완전히 빠진 것) 처리
        - Null, NAN, '', Empty 등 값이 없는 컬럼 제거 및 유용한 값으로 치환
        - Null, NAN은 통계를 위해 계산하면 계산된 값도 null로 바꿔버림
        - (필수!) 숫자로 된 컬럼 값에는 결측치를 제거/변경 해줘야 함

- 데이터 전처리가 빅데이터 분석 전체 프로젝트에 거의 50% 정도를 차지

In [ ]:
# 재확인
df_seoulStore

,매장 코드,매장명,매장유형,주소,위도,경도
0,1190,동명대DT,generalDT,부산광역시 남구 신선로 423 (용당동),35.12311959047579,129.09901642703608
1,2235,부산유엔공원,general,부산광역시 남구 유엔로 200 (대연동),35.1299808,129.0980971
2,683,부산대연역,general,"부산광역시 남구 수영로 240-1, 1층 (대연동)",35.134998534278104,129.0930603381513
3,1795,부산대연못골,general,부산광역시 남구 못골로 87 (대연동),35.13609516236527,129.09191736599408
4,248,경성대,general,부산광역시 남구 수영로 312 (대연동),35.137345553736964,129.10063775537583
...,...,...,...,...,...,...
632,838,사가정역,general,서울특별시 중랑구 면목로 310,37.579594,127.087966
633,493,상봉역,general,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
634,1668,묵동,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
635,2002,양원역,general,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


In [ ]:
# 전체 데이터에서 주소에 부산광역시가 포함된 여부 출력 => True면 부산광역시가 포함되어 있음
df_seoulStore['주소'].str.contains('부산광역시')  

0       True
1       True
2       True
3       True
4       True
       ...  
632    False
633    False
634    False
635    False
636    False
Name: 주소, Length: 637, dtype: bool

In [ ]:
# 부산광역시가 포함된 index 출력

del_list = df_seoulStore[df_seoulStore['주소'].str.contains('부산광역시')].index


In [ ]:
# 서울이 아닌 인덱스 값 삭제(drop)
# 1. drop() 함수로 변경된 값을 (df_seoulStore2 변수에) 재할당
df_seoulStore2 = df_seoulStore.drop(del_list)
df_seoulStore2

,매장 코드,매장명,매장유형,주소,위도,경도
23,1509,역삼아레나빌딩,general,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
24,1434,논현역사거리,general,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
25,1595,신사역성일빌딩,general,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057
26,1527,국기원사거리,general,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
27,1468,대치재경빌딩,general,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...
632,838,사가정역,general,서울특별시 중랑구 면목로 310,37.579594,127.087966
633,493,상봉역,general,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
634,1668,묵동,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
635,2002,양원역,general,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


- drop() 등 몇가지 함수는 값을 적용하는게 아닌 값이 바뀐 걸 PreView 형태로 보여주는 함수임
- 값을 변경하려면, 새로운 변수나 같은 변수에 다시 할당하거나 속성 중 inplace=True로 실행

In [ ]:
df_seoulStore

,매장 코드,매장명,매장유형,주소,위도,경도
0,1190,동명대DT,generalDT,부산광역시 남구 신선로 423 (용당동),35.12311959047579,129.09901642703608
1,2235,부산유엔공원,general,부산광역시 남구 유엔로 200 (대연동),35.1299808,129.0980971
2,683,부산대연역,general,"부산광역시 남구 수영로 240-1, 1층 (대연동)",35.134998534278104,129.0930603381513
3,1795,부산대연못골,general,부산광역시 남구 못골로 87 (대연동),35.13609516236527,129.09191736599408
4,248,경성대,general,부산광역시 남구 수영로 312 (대연동),35.137345553736964,129.10063775537583
...,...,...,...,...,...,...
632,838,사가정역,general,서울특별시 중랑구 면목로 310,37.579594,127.087966
633,493,상봉역,general,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
634,1668,묵동,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
635,2002,양원역,general,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


In [ ]:
#2. inplace=True 사용
df_seoulStore.drop(del_list, inplace=True)

In [ ]:
df_seoulStore

,매장 코드,매장명,매장유형,주소,위도,경도
23,1509,역삼아레나빌딩,general,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
24,1434,논현역사거리,general,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
25,1595,신사역성일빌딩,general,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057
26,1527,국기원사거리,general,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
27,1468,대치재경빌딩,general,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...
632,838,사가정역,general,서울특별시 중랑구 면목로 310,37.579594,127.087966
633,493,상봉역,general,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
634,1668,묵동,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
635,2002,양원역,general,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


In [ ]:
# DF 인덱스가 0이 아닌 값으로 시작, 인덱스 초기화
## drop=True는 인덱스 삭제, inplace=True 값 변경 완전적용
df_seoulStore.reset_index(inplace=True, drop=True)

In [ ]:
# 단순히 컬럼을 삭제
## axis=0(default) 행삭제, axis=1 열삭제
df_seoulStore.drop('index', axis=1)

,매장 코드,매장명,매장유형,주소,위도,경도
0,1509,역삼아레나빌딩,general,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,1434,논현역사거리,general,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,1595,신사역성일빌딩,general,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057
3,1527,국기원사거리,general,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,1468,대치재경빌딩,general,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...
609,838,사가정역,general,서울특별시 중랑구 면목로 310,37.579594,127.087966
610,493,상봉역,general,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
611,1668,묵동,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
612,2002,양원역,general,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


In [ ]:
df_seoulStore.to_csv('./data/스타벅스_서울매장정보.csv', encoding='utf-8')

#### 전국 시군구 위경도에서 서울만 추출

In [ ]:
# 엑셀파일 리드 라이브러리 설치
!pip install openpyxl

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------  245.8/250.9 kB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 3.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# 전국 시군구 위경도 엑셀파일 로드
df_koreaManicipality = pd.read_excel('./data/전국_시군구_위경도.xlsx')

In [ ]:
df_koreaManicipality.tail()

,docity,do,city,longitude,latitude
290,충청충주시,충청,충주시,127.928144,36.988181
291,충청태안군,충청,태안군,126.299975,36.742667
292,충청한누리대로,충청,한누리대로,127.289926,36.485450
293,충청홍성군,충청,홍성군,126.662908,36.598361
294,충청대전시,충청,대전시,127.384862,36.350630


In [ ]:
#서울구별 데이터 필터링
seoulList = df_koreaManicipality['do'] == '서울'

In [ ]:
df_koreaManicipality = df_koreaManicipality[seoulList]

In [ ]:
df_koreaManicipality.reset_index(drop=True, inplace=True)
df_koreaManicipality

,docity,do,city,longitude,latitude
0,서울강남구,서울,강남구,127.049556,37.514575
1,서울강동구,서울,강동구,127.125864,37.527367
2,서울강북구,서울,강북구,127.027719,37.636956
3,서울강서구,서울,강서구,126.851675,37.548156
4,서울관악구,서울,관악구,126.953844,37.475386
5,서울광진구,서울,광진구,127.084533,37.535739
6,서울구로구,서울,구로구,126.889597,37.492650
7,서울금천구,서울,금천구,126.904197,37.449108
8,서울노원구,서울,노원구,127.058389,37.651461
9,서울도봉구,서울,도봉구,127.049522,37.665833


In [ ]:
# 서울 구별 위치값 저장
df_koreaManicipality.to_csv('./data/서울구별위치.csv', encoding='utf-8')

In [ ]:
# DF 정보확인
## 데이터 결측치, 타입 확인
## object == string
df_seoulStore.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   614 non-null    int64 
 1   매장 코드   614 non-null    object
 2   매장명     614 non-null    object
 3   매장유형    614 non-null    object
 4   주소      614 non-null    object
 5   위도      614 non-null    object
 6   경도      614 non-null    object
dtypes: int64(1), object(6)
memory usage: 33.7+ KB


In [ ]:
df_koreaManicipality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   docity     25 non-null     object 
 1   do         25 non-null     object 
 2   city       25 non-null     object 
 3   longitude  25 non-null     float64
 4   latitude   25 non-null     float64
dtypes: float64(2), object(3)
memory usage: 1.1+ KB
